In [ ]:
import os
import os.path as osp
import glob

data_path = ''
save_path = ''

def get_video_paths(data_path):
    vps = sorted(
        glob.glob(osp.join(data_path, 'MEAD', 'M009', 'video', '*', '*', '*') + '/*.mp4'))
    return vps 


def extract_frames(video_path, save_path, pid, angle, emo, lvl, gt_size=1024, scale_factor=16):
    filename, extension = osp.splitext(osp.basename(video_path))

    os.makedirs(osp.join(save_path, 'MEAD', pid, 'frames', angle, emo, lvl, filename, 'hq_raw'), exist_ok=True)
    os.makedirs(osp.join(save_path, 'MEAD', pid, 'frames', angle, emo, lvl, filename, 'hq'), exist_ok=True)
    os.makedirs(osp.join(save_path, 'MEAD', pid, 'frames', angle, emo, lvl, filename, 'lq'), exist_ok=True)

    video_reader = mmcv.VideoReader(video_path)

    for i, frame in enumerate(video_reader):
        hq_raw = np.flip(frame, axis=2) / 255
        h, w, _ = hq_raw.shape
        hq = hq_raw[(h-gt_size)//2:(h-gt_size)//2 + gt_size, (w-gt_size)//2:(w-gt_size)//2 + gt_size, :]
        
        lq = np.clip(resize(hq, scale_factors=1/scale_factor), 0, 1)

        hq_raw = Image.fromarray((hq_raw * 255).astype(np.uint8))
        hq = Image.fromarray((hq * 255).astype(np.uint8))
        lq = Image.fromarray((lq * 255).astype(np.uint8))

        
        hq_raw.save(osp.join(save_path, 'MEAD', pid, 'frames', angle, emo, lvl, filename, 'hq_raw', f'{i:03}.png'))
        hq.save(osp.join(save_path, 'MEAD', pid, 'frames', angle, emo, lvl, filename, 'hq', f'{i:03}.png'))
        lq.save(osp.join(save_path, 'MEAD', pid, 'frames', angle, emo, lvl, filename, 'lq', f'{i:03}.png'))


video_paths = get_video_paths(data_path=data_path)
for vp in video_paths:
    pid, _, angle, emo, lvl, filename = osp.normpath(vp).split(osp.sep)[-6:]

    if lvl == 'level_3' and filename == '001.mp4':
        extract_frames(vp, save_path, pid, angle, emo, lvl)